<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/Step2_LearningToRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Work in 2 process:
1. Rank: The fit column is empty, and we should calculate the similarity of the search term against the job titles of the candidates using cosine similarity.
2. Design and Learning to rank: We need labels for the 2nd project.
- Pick a few candidates and manually overwrite their fit score (cosine_similarity). I should pick one among the word embedding methods I've gon through so far.
- Train a learning to rank model in particular, 'RankNet' in PyTorch (<- they have many examples on github in code). Rank Net use pair-wise ranking, I need to transform fit score by selecting to candidates randomly and comparing their fit scores if one of the fit score is larger than the pair, it is transformed as 1. if it's smaller than the other pair, the label becomes 0, indicating that one candidate should be ranked above the other candidates.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Necessary Package Installation

In [ ]:
# !pip install wordcloud
!pip install nltk
!pip install gensim
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

### Open CSV

In [ ]:
# open data
import numpy as np
import pandas as pd

# text data vectorization and word embedding
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sentence_transformers import SentenceTransfomrer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
tfidf_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/tfidf_df.csv')
w2v_cbow_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
w2v_sg_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
glove_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/glove_df.csv')
fasttext_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/fasttext_df.csv')
stb_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/stb_df.csv')
w2v_sg_df

,Unnamed: 0,id,job_title,location,connection,fit,cosine_similarity_w2v_cbow,rank_w2v_cbow
0,5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,1
1,59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,2
2,48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,3
3,35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,4
4,23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,5
...,...,...,...,...,...,...,...,...
99,90,91,Lead Official at Western Illinois University,Greater Chicago Area,39,NaN,-0.189587,100
100,33,34,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,-0.189587,101
101,34,35,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,-0.189587,102
102,22,23,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,-0.189587,103


In [ ]:
fasttext_df

,id,job_title,location,connection,fit,cosine_similarity_ft,rank_ft
0,53,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN,0.983347,1
1,26,HR Senior Specialist,San Francisco Bay Area,500+,NaN,0.983347,2
2,88,Human Resources Management Major,"Milpitas, California",18,NaN,0.983347,3
3,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,NaN,0.983347,4
4,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,NaN,0.983347,5
...,...,...,...,...,...,...,...
99,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,100
100,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,101
101,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,NaN,0.413387,102
102,34,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,0.413387,103


In [ ]:
# print("Dataset check:\n", fasttext_df)
print("Are there NaN values?", fasttext_df.isna().sum())
fasttext_df['fit'] = fasttext_df['fit'].fillna(0.5)

Are there NaN values? id                        0
job_title                 0
location                  0
connection                0
fit                     104
cosine_similarity_ft      0
rank_ft                   0
dtype: int64


### LearningToRank

1. Cosine Similarity as fit

In [ ]:
fasttext_df.fit = fasttext_df.cosine_similarity_ft
# fasttext_df.fit
# fasttext_df

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# Step 1: Generate Pairwise Data
pairwise_data = []
for _ in range(1000):  # Generate 1000 random pairs
    a, b = random.sample(range(len(fasttext_df)), 2)
    fit_a, fit_b = fasttext_df.loc[a, 'fit'], fasttext_df.loc[b, 'fit']
    label = 1 if fit_a > fit_b else 0  # Transform fit scores into 1 or 0
    pairwise_data.append((a, b, label))

# Convert to tensor format
pairwise_data = torch.tensor(pairwise_data, dtype=torch.float32)
# print(pairwise_data)

In [ ]:
# Step 2: Define RankNet Model (Without BatchNorm)
class RankNet(nn.Module):
    def __init__(self, input_size):
        super(RankNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.activation = nn.LeakyReLU(0.01)  # Prevent dead neurons

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        return self.fc3(x)  # No Sigmoid

# Dummy Input Size
input_size = 1  # Using fit scores, can be more features
ranknet = RankNet(input_size)

# Apply Xavier Weight Initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
ranknet.apply(init_weights)

# Step 3: Training RankNet
optimizer = optim.Adam(ranknet.parameters(), lr=0.0005)
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss


# Training
for epoch in range(500):
    total_loss = 0
    for a, b, label in pairwise_data:
        # Extract candidate features (only 'fit' scores here)
        input_a = torch.tensor([fasttext_df.loc[int(a), 'fit']]).float().unsqueeze(0).unsqueeze(1)  # Ensure 2D
        input_b = torch.tensor([fasttext_df.loc[int(b), 'fit']]).float().unsqueeze(0).unsqueeze(1)  # Ensure 2D

        # Forward pass
        output_a = ranknet(input_a)
        output_b = ranknet(input_b)

        # Compute probability with NaN Prevention
        pred = output_a - output_b.clamp(min=1e-7, max=1-1e-7)
        target = torch.tensor([label], dtype=torch.float32)

        # Compute loss
        loss = loss_fn(pred.view(-1), target)

        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # # Debug: Print Gradients
        # for name, param in ranknet.named_parameters():
        #   print(f"{name}: {param.grad}")

        # Clip gradients to prevent explosion
        torch.nn.utils.clip_grad_norm_(ranknet.parameters(), max_norm=1)
        optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

Epoch 0, Loss: 512.366612304002
Epoch 50, Loss: 728.3020269828849
Epoch 100, Loss: 695.8546441541985
Epoch 150, Loss: 681.5474870749749
Epoch 200, Loss: 673.7100497279316
Epoch 250, Loss: 664.7594197522849
Epoch 300, Loss: 642.8918944569305
Epoch 350, Loss: 645.2178435423411
Epoch 400, Loss: 650.9613181808963
Epoch 450, Loss: 653.3732993518934


In [ ]:
# Step 4: Ranking Candidates
def rank_candidates(df, model):
    predictions = []
    for i in range(len(df)):
        input_tensor = torch.tensor([df.loc[i, 'fit']]).float().unsqueeze(0).unsqueeze(1)  # Ensure 2D input
        score = model(input_tensor).item()

        # Apply sigmoid to normalize scores between 0 and 1
        normalized_score = torch.sigmoid(torch.tensor(score)).item()

        predictions.append((df.loc[i, 'job_title'], normalized_score))

    # Sort by predicted score
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions

# Ranking candidates
ranked_candidates = rank_candidates(fasttext_df, ranknet)
print("\nFinal Ranking:")
for rank, (title, score) in enumerate(ranked_candidates, 1):
    print(f"{rank}. {title} (Score: {score:.4f})")


Final Ranking:
1. Seeking Human Resources HRIS and Generalist Positions (Score: 0.9875)
2. HR Senior Specialist (Score: 0.9875)
3. Human Resources Management Major (Score: 0.9875)
4. Human Resources Specialist at Luxottica (Score: 0.9875)
5. Information Systems Specialist and Programmer with a love for data and organization. (Score: 0.9875)
6. HR Manager at Endemol Shine North America (Score: 0.9875)
7. Undergraduate Research Assistant at Styczynski Lab (Score: 0.9875)
8. Seeking employment opportunities within Customer Service or Patient Care (Score: 0.9872)
9. RRP Brand Portfolio Executive at JTI (Japan Tobacco International) (Score: 0.9872)
10. Native English Teacher at EPIK (English Program in Korea) (Score: 0.9872)
11. Student at Humber College and Aspiring Human Resources Generalist (Score: 0.9872)
12. Student at Humber College and Aspiring Human Resources Generalist (Score: 0.9872)
13. Director Human Resources  at EY (Score: 0.9806)
14. Student at Humber College and Aspiring Hu

In [ ]:
for name, param in ranknet.named_parameters():
  print(f"{name}: {param.data.mean().item()}")

fc1.weight: 0.02470952644944191
fc1.bias: -0.06718002259731293
fc2.weight: -0.21676215529441833
fc2.bias: 0.1388578861951828
fc3.weight: -0.23177474737167358
fc3.bias: 0.4765144884586334
